Encodes Am tags from ledger into filenames

only include expl and/or enc tagged

name format:

vol_chap-date-nat-enc-expl-title-pagerange
enc_full enc_part enc_min enc_none
expl_y expl_n

underscores within features, dashes between features

for entry in ledger:
    if enc OR expl:
        locate file by vol-chap
        save file in new dir with new name

In [45]:
old_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat/'
new_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_mod/'

import csv
import os
import re


with open('text-data/AMledger.csv', mode = 'r') as ledger_csv:
    ledger = csv.DictReader(ledger_csv)
    for row in ledger:
        #print('starting row')
        if 'v' in row['expl'] or 'v' in row['enc'] or 'v' in row['enc_min']:
            #print('row of interest')
            # if entry addresses travel or cultural encounter:
            filelist = os.scandir(old_path)
            for entry in filelist:
                #print('scanning' , entry.name)
                #print ( 'condition checks: ', row['vol'].zfill(2), re.search(r'_(.*?)_', entry.name).group(1), row['vol'].zfill(2) == entry.name[:2], row['chap'] == re.search(r'_(.*?)_', entry.name).group(1))
                if row['vol'].zfill(2) == entry.name[:2] and (row['chap'].zfill(2) == str(float(re.search(r'_(.*?)_', entry.name).group(1))) or row['chap'].zfill(2) == re.search(r'_(.*?)_', entry.name).group(1)):
                    #print(entry.name, 'passed')
                    # find the relevant text file
                    with open(entry.path, 'r', encoding="utf8") as f:
                        text = f.read()
                        expl = 'expl_y' if row['expl'] == 'v' else 'expl_n'
                        if row['enc'] == 'v' and row['expl'] != 'v':
                            enc = 'enc_full'
                        elif row['enc'] == 'v':
                            enc = 'enc_part'
                        elif row['enc_min'] == 'v':
                            enc = 'enc_min'
                        else:
                            enc = 'enc_none'
                        # determine name factors
                        name = row['vol'].zfill(2) + '_' + row['chap'].zfill(2) + '-' + row['date'] + '-' + row['nat'] + '-' + enc + '-' + expl + '-' + row['title'] + '-' + row['pages'].replace('-', '_') + '.txt'
                        # string formatted filename
                        with open((new_path + '/' + name), 'w', encoding="utf8") as fw:
                            fw.write(text)
                            #print('text outputted')
                    #break
                            
                            

In [ ]:
if row['vol'].zfill(2) == entry.name[:2] and row['chap'].zfill(2) == re.search(r'_(.*?)_', entry.name).group(1):

In [41]:
test = '09_01.1_AMER_1564_The_voiage_of_captaine_Rene_Laudonniere_to_Florida_1564_where_he_fortified_and_inhabited_two_summers_pp.001-035'
print(str(float(re.search(r'_(.*?)_', test).group(1))))
#"1.1".zfill(2)



1.1
